In [3]:
import pandas as pd 
from urllib.parse import urlencode
import csv
import numpy as np 
import re, itertools
from top2vec  import Top2Vec
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [14]:
initialDF_midCovid= pd.read_csv('Hansard_mid_covid.csv', sep=',', encoding='utf-16-le')
initialDF_midCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1)
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_midCovid['Text'] = initialDF_midCovid['Text'].astype(str).str.lower()
initialDF_midCovid['Text'] = initialDF_midCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['Text'].apply(word_tokenize)
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_midCovid['text_tokens'] = initialDF_midCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))


Preprocessing the text column using nltk library begins

In [ ]:

# !python -m pip install top2vec[sentence_encoders]


In [28]:
midCovid_unguidedModel = Top2Vec(initialDF_midCovid['text_tokens'].values, embedding_model='universal-sentence-encoder')
midCovid_unguidedModel.topic_words

2023-03-02 02:53:55,102 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
c:\Users\Kriti Kapoor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2023-03-02 02:53:55,500 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2023-03-02 02:54:25,806 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2023-03-02 02:54:29,404 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2023-03-02 02:54:38,740 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2023-03-02 02:54:38,794 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


array([['worker', 'immigration', 'nafta', 'temporary', 'farmer',
        'pandemic', 'quarantine', 'employer', 'foreign', 'labour',
        'colleague', 'working', 'emergency', 'trafficking', 'work',
        'farm', 'agriculture', 'term', 'another', 'issue', 'economy',
        'wage', 'process', 'long', 'student', 'last', 'indigenous',
        'agreement', 'part', 'government', 'economic', 'province',
        'system', 'even', 'important', 'essential', 'security', 'broken',
        'question', 'housing', 'quebec', 'human', 'service', 'canadian',
        'risk', 'case', 'minister', 'plan', 'benefit', 'problem'],
       ['immigration', 'total', 'nafta', 'indigenous', 'economy',
        'liberal', 'risk', 'labour', 'pandemic', 'canadian', 'broken',
        'province', 'month', 'response', 'farmer', 'quebec', 'state',
        'last', 'agriculture', 'industry', 'speech', 'issue', 'however',
        'world', 'supply', 'amount', 'billion', 'still', 'increase',
        'including', 'thing', 'c

In [ ]:
words, word_scores = midCovid_unguidedModel.similar_words(keywords=["foreign"], keywords_neg=[], num_words=40)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

In [29]:
midCovid_unguidedModel.hierarchical_topic_reduction(num_topics =1)
midCovid_unguidedModel.topic_words_reduced

array([['immigration', 'worker', 'nafta', 'pandemic', 'farmer', 'labour',
        'temporary', 'quarantine', 'economy', 'colleague', 'foreign',
        'agriculture', 'indigenous', 'farm', 'trafficking', 'issue',
        'working', 'term', 'another', 'last', 'wage', 'province',
        'emergency', 'long', 'broken', 'risk', 'employer', 'canadian',
        'government', 'economic', 'work', 'student', 'quebec', 'even',
        'agreement', 'process', 'state', 'minister', 'system', 'part',
        'question', 'plan', 'problem', 'important', 'case', 'support',
        'benefit', 'solution', 'crisis', 'total']], dtype='<U11')

Now let's check pre-covid 

In [4]:
# initialDF_preCovid= pd.read_csv('Hansard_preCovid.csv', sep=',', encoding='utf-16-le', error_bad_lines = False) 
# # may need to change separator to include skipped lines
# initialDF_preCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1) 

newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

In [30]:
preCovid_unguidedModel = Top2Vec(initialDF_preCovid['text_tokens'].values, embedding_model='universal-sentence-encoder')

2023-03-02 02:56:12,005 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
c:\Users\Kriti Kapoor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2023-03-02 02:56:12,796 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2023-03-02 02:56:38,131 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2023-03-02 02:56:40,965 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2023-03-02 02:56:47,111 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2023-03-02 02:56:47,186 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [31]:
preCovid_unguidedModel.topic_words

array([['immigration', 'immigrant', 'worker', 'nafta', 'labour',
        'refugee', 'citizenship', 'resident', 'caregiver', 'visa',
        'citizen', 'border', 'applicant', 'shortage', 'employment',
        'disability', 'indigenous', 'employer', 'temporary', 'process',
        'motion', 'parliamentary', 'initiative', 'wage', 'agreement',
        'term', 'employee', 'skilled', 'concern', 'permanent', 'demand',
        'foreign', 'ceta', 'significant', 'conservative', 'working',
        'claim', 'system', 'issue', 'consultant', 'canadian', 'appeal',
        'asylum', 'matter', 'liberal', 'backlog', 'provision', 'work',
        'economy', 'minister'],
       ['previous', 'shortage', 'immigration', 'debate', 'development',
        'nafta', 'total', 'indigenous', 'backlog', 'point', 'liberal',
        'filipino', 'economy', 'canadian', 'claim', 'province',
        'provincial', 'hope', 'believe', 'month', 'matter', 'enough',
        'fact', 'construction', 'global', 'broken', 'border', 'f

In [32]:
preCovid_unguidedModel.hierarchical_topic_reduction(num_topics =1)
preCovid_unguidedModel.topic_words_reduced

array([['immigration', 'nafta', 'immigrant', 'labour', 'shortage',
        'refugee', 'worker', 'indigenous', 'border', 'citizenship',
        'visa', 'applicant', 'previous', 'citizen', 'canadian',
        'backlog', 'claim', 'temporary', 'liberal', 'agreement',
        'matter', 'economy', 'demand', 'issue', 'conservative',
        'initiative', 'province', 'caregiver', 'disability', 'term',
        'development', 'resident', 'significant', 'concern', 'debate',
        'process', 'ceta', 'construction', 'parliamentary', 'wage',
        'employment', 'foreign', 'committee', 'long', 'motion',
        'project', 'consultant', 'permanent', 'minister', 'system']],
      dtype='<U13')

In [22]:
alpha, beta = preCovid_unguidedModel.similar_words(keywords=["human"], keywords_neg=[], num_words=40)
for word, score in zip(alpha, beta):
    print(f"{word} {score}")

ValueError: 'human' has not been learned by the model so it cannot be searched.

In [ ]:
topic_words, topic_scores

post covid

In [33]:
#Converting csv to txt file for ease of processing
newline = []

textstring = ''
with open('Hansard_postCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_postCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_postCovid['Text'] = initialDF_postCovid['Text'].astype(str).str.lower()
initialDF_postCovid['Text'] = initialDF_postCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_postCovid['text_tokens'] = initialDF_postCovid['Text'].apply(word_tokenize)
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))


In [34]:

postCovid_unguidedModel = Top2Vec(initialDF_postCovid['text_tokens'].values, embedding_model='universal-sentence-encoder')


2023-03-02 02:58:08,957 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
c:\Users\Kriti Kapoor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2023-03-02 02:58:09,306 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2023-03-02 02:58:32,110 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2023-03-02 02:58:34,599 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2023-03-02 02:58:54,088 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2023-03-02 02:58:54,169 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [35]:

postCovid_unguidedModel.topic_words

array([['in', 'xa', 'the', 'made', 'three', 'well', 'point', 'already',
        'right', 'year', 'thing', 'question', 'help', 'still', 'we',
        'another', 'including', 'done', 'sure', 'last', 'great', 'even',
        'first', 'nothing', 'much', 'ircc', 'would', 'part', 'hard',
        'bloc', 'problem', 'issue', 'world', 'mean', 'come', 'need',
        'think', 'back', 'across', 'know', 'provide', 'going', 'take',
        'good', 'forward', 'number', 'better', 'want', 'fact', 'example'],
       ['immigration', 'immigrant', 'worker', 'labour', 'shortage',
        'resident', 'quebecois', 'employer', 'temporary', 'economy',
        'residency', 'colleague', 'foreign', 'working', 'motion', 'work',
        'term', 'permanent', 'waiting', 'economic', 'issue', 'process',
        'backlog', 'quebec', 'pandemic', 'farmer', 'application',
        'election', 'agriculture', 'province', 'government', 'problem',
        'important', 'housing', 'another', 'long', 'part', 'liberal',
        'ca

In [36]:
postCovid_unguidedModel.hierarchical_topic_reduction(num_topics =1)
postCovid_unguidedModel.topic_words_reduced

array([['xa', 'point', 'in', 'the', 'made', 'well', 'three', 'question',
        'year', 'already', 'another', 'thing', 'right', 'still', 'last',
        'including', 'help', 'even', 'done', 'sure', 'issue', 'nothing',
        'we', 'problem', 'part', 'first', 'great', 'bloc', 'hard',
        'much', 'motion', 'canadian', 'think', 'number', 'provide',
        'would', 'ircc', 'long', 'come', 'supply', 'world', 'need',
        'application', 'solution', 'case', 'increase', 'mean', 'example',
        'know', 'term']], dtype='<U11')

In [23]:
alpha, beta = postCovid_unguidedModel.similar_words(keywords=["human"], keywords_neg=[], num_words=40)
for word, score in zip(alpha, beta):
    print(f"{word} {score}")

ValueError: 'human' has not been learned by the model so it cannot be searched.